# Prepare Malaya-Speech Dataset

We want our model able to understand Bahasa and local english slang (Manglish).

### Download data

Run command below to download all data,

```bash
wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-manglish.tar.gz
tar -xf semisupervised-manglish.tar.gz
wget https://cdn.commonvoice.mozilla.org/cv-corpus-5.1-2020-06-22/id.tar.gz
tar -zxf id.tar.gz
wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-malay.tar.gz
tar -xf semisupervised-malay.tar.gz
```

Total samples length,

1. Malay, ~93 hours.
2. Manglish, ~107 hours.
3. Indonesian, ~4 hours.

All these data are not really clean.

### Read data

In [1]:
import pandas as pd
import malaya_speech
import malaya_speech.train as train

In [2]:
df = pd.read_csv('cv-corpus-5.1-2020-06-22/id/validated.tsv', sep = '\t')
df = df[(df['sentence'].str.len() > 5) & (df['sentence'].str.count(' ') > 0)]
print(df.shape)

(7490, 10)


In [3]:
id_commonvoice = []
for i in range(len(df)):
    p = f"cv-corpus-5.1-2020-06-22/id/clips/{df['path'].iloc[i]}"
    id_commonvoice.append((p, df['sentence'].iloc[i]))

len(id_commonvoice)

7490

In [4]:
from glob import glob

malay = glob('semisupervised-malay/output-wav/*.wav')
manglish = glob('semisupervised-manglish/output-wav/*.wav')
len(malay), len(manglish)

(57895, 65277)

In [5]:
import unicodedata
import re


def preprocessing_text(string, reject = ',.!?'):
    for c in reject:
        string = string.replace(c, ' ')
        
    string = unicodedata.normalize('NFC', string.lower())
    return re.sub(r'[ ]+', ' ', string).strip()

In [6]:
from tqdm import tqdm

malays = []
for i in tqdm(malay):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        malays.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 57895/57895 [00:01<00:00, 36185.03it/s]


In [7]:
manglishs = []
for i in tqdm(manglish):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        manglishs.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 65277/65277 [00:01<00:00, 37518.22it/s]


In [8]:
audios = id_commonvoice + malays + manglishs
audios, texts = zip(*audios)

In [9]:
from pydub import AudioSegment
import numpy as np

def mp3_to_wav(file, sr = 16000):
    audio = AudioSegment.from_file(file)
    audio = audio.set_frame_rate(sr).set_channels(1)
    sample = np.array(audio.get_array_of_samples())
    return malaya_speech.astype.int_to_float(sample), sr

### Change into TFRecord

This is not necessary step, we recommend to use yield iterator to train the model, but we also can save our data into TFRecord to speed up data pipelines. To do that, we need to create a yield iterator.

In [10]:
def generator(maxlen = 16):
    for i in tqdm(range(len(audios))):
        try:
            if '.mp3' in audios[i]:
                wav_data, sr = mp3_to_wav(audios[i])
            else:
                wav_data, sr = malaya_speech.load(audios[i])
                
            if (len(wav_data) / sr) > maxlen:
                print(f'skipped {audios[i]}')

            yield {
                'waveforms': wav_data.tolist(),
                'waveform_lens': [len(wav_data)],
                'targets': malaya_speech.char.encode(texts[i]),
                'raw_transcript': [texts[i]],
            }
        except Exception as e:
            print(e)
            
generator = generator()

In [11]:
import os
import tensorflow as tf

os.system('rm bahasa-asr/data/*')
DATA_DIR = os.path.expanduser('bahasa-asr/data')
tf.gfile.MakeDirs(DATA_DIR)

#### Define shards

Like we defined below,

```python
shards = [{'split': 'train', 'shards': 1000}, {'split': 'dev', 'shards': 1}]
```

In [12]:
shards = [{'split': 'train', 'shards': 1000}, {'split': 'dev', 'shards': 1}]

#### Save to TFRecord

Just pass yield iterator to malaya_speech.train_prepare_dataset,

```python
def prepare_dataset(
    generator,
    data_dir: str,
    shards: List[Dict],
    prefix: str = 'dataset',
    shuffle: bool = True,
    already_shuffled: bool = False,
):
```

In [13]:
train.prepare_dataset(generator, DATA_DIR, shards, prefix = 'bahasa-asr')

  0%|          | 0/130662 [00:00<?, ?it/s]


INFO:tensorflow:Generating case 0.


  6%|▌         | 7742/130662 [10:49<31:27, 65.11it/s]  

skipped semisupervised-malay/output-wav/Bila-Mata-Bertentang-Mata-❤️-_-Ustaz-Mohd-Lukmanul-Hakim-Wm-8ZeLyux0.mp3-part-312.wav


  7%|▋         | 8516/130662 [11:02<34:59, 58.19it/s]

...

100%|█████████▉| 130518/130662 [45:29<00:02, 60.12it/s]

skipped semisupervised-manglish/output-wav/Mamak-Sessions-Podcast---How-Do-You-Handle-Long-Distance-Relationships-Do-they-WORK-m7mwwtsblIs.mp3-part-5.wav


100%|██████████| 130662/130662 [45:32<00:00, 47.82it/s]


INFO:tensorflow:Generated 130662 Examples
INFO:tensorflow:Shuffling data...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`



INFO:tensorflow:Data shuffled.
